In [3]:

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import nltk
nltk.download('stopwords')



# 2️⃣ ملفات التدريب والاختبار
train_files = [
    r"C:\Users\PC\Desktop\ai\train_Arabic_tweets_negative_20190413.tsv",
    r"C:\Users\PC\Desktop\ai\train_Arabic_tweets_positive_20190413.tsv"
]
test_files = [
    r"C:\Users\PC\Desktop\ai\test_Arabic_tweets_negative_20190413.tsv",
    r"C:\Users\PC\Desktop\ai\test_Arabic_tweets_positive_20190413.tsv"
]



# 3️⃣ دالة تنظيف النصوص
stemmer = ISRIStemmer()
arabic_stopwords = set(stopwords.words('arabic'))

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www.\S+", "", text)   # إزالة الروابط
    text = re.sub(r"@\w+", "", text)               # إزالة @
    text = re.sub(r"#", "", text)                  # إزالة #
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text) # إزالة الرموز والأرقام
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    words = [stemmer.stem(w) for w in text.split() if w not in arabic_stopwords]
    return " ".join(words)




# 4️⃣ قراءة الملفات وتنظيفها
def read_and_clean(files):
    all_data = []
    for file in files:
        df = pd.read_csv(file, sep="\t", header=None, names=["label", "text"])
        df["label"] = df["label"].map({"neg": 0, "pos": 1})
        df["clean_text"] = df["text"].apply(clean_text)
        all_data.append(df[["clean_text", "label"]])
    return pd.concat(all_data, ignore_index=True)

train_df = read_and_clean(train_files)
test_df = read_and_clean(test_files)




# 5️⃣ توازن الفئات (oversampling)
df_majority = train_df[train_df.label==0]
df_minority = train_df[train_df.label==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,     
                                 n_samples=len(df_majority),    
                                 random_state=42)

train_df_balanced = pd.concat([df_majority, df_minority_upsampled])




# 6️⃣ TF-IDF محسّن
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1,2))
X_train = vectorizer.fit_transform(train_df_balanced["clean_text"])
X_test = vectorizer.transform(test_df["clean_text"])
y_train = train_df_balanced["label"]
y_test = test_df["label"]




# 7️⃣ تدريب شبكة عصبية أكبر ومحسنة
mlp = MLPClassifier(
    hidden_layer_sizes=(150, 50),
    max_iter=50,
    random_state=42,
    verbose=True,
    learning_rate_init=0.001
)
mlp.fit(X_train, y_train)




# 8️⃣ اختبار النموذج
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\nدقة النموذج على الاختبار:", round(accuracy, 4))
print("\nتقرير التصنيف:")
print(classification_report(y_test, y_pred, target_names=["Negative", "Positive"]))           



# # حفظ النموذج و TF-IDF Vectorizer
# import joblib
# joblib.dump(mlp, "mlp_model.pkl")
# joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
# print("تم حفظ النموذج وVectorizer جاهزين للاستخدام في الويب!")


# mlp = joblib.load("mlp_model.pkl")
# vectorizer = joblib.load("tfidf_vectorizer.pkl")



# 9️⃣ واجهة تجربة التغريدات الجديدة
while True:
    tweet = input("\nادخل تغريدة جديدة (اكتب 'exit' للخروج):\n>>> ")
    if tweet.lower() == "exit":
        print("تم إنهاء البرنامج.")
        break
    tweet_clean = clean_text(tweet)
    tweet_vector = vectorizer.transform([tweet_clean])
    pred = mlp.predict(tweet_vector)[0]
    print("➡️ التغريدة إيجابية" if pred == 1 else "➡️ التغريدة سلبية")







[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Iteration 1, loss = 0.48892529
Iteration 2, loss = 0.34278636
Iteration 3, loss = 0.26010966
Iteration 4, loss = 0.17342444
Iteration 5, loss = 0.10914229
Iteration 6, loss = 0.07412748
Iteration 7, loss = 0.05882393
Iteration 8, loss = 0.05119724
Iteration 9, loss = 0.04763446
Iteration 10, loss = 0.04500683
Iteration 11, loss = 0.04345088
Iteration 12, loss = 0.04219227
Iteration 13, loss = 0.04127707
Iteration 14, loss = 0.04128120
Iteration 15, loss = 0.04043555
Iteration 16, loss = 0.03968348
Iteration 17, loss = 0.03893305
Iteration 18, loss = 0.03932761
Iteration 19, loss = 0.03908522
Iteration 20, loss = 0.03814079
Iteration 21, loss = 0.03791258
Iteration 22, loss = 0.03768116
Iteration 23, loss = 0.03756063
Iteration 24, loss = 0.03744141
Iteration 25, loss = 0.03726987
Iteration 26, loss = 0.03696747
Iteration 27, loss = 0.03646602
Iteration 28, loss = 0.03642213
Iteration 29, loss = 0.03673453
Iteration 30, loss = 0.03627997
Iteration 31, loss = 0.03638670
Iteration 32, los

C:\Users\PC\anaconda3\anac\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 50, loss = 0.03318481

دقة النموذج على الاختبار: 0.774

تقرير التصنيف:
              precision    recall  f1-score   support

    Negative       0.74      0.85      0.79      5768
    Positive       0.82      0.70      0.75      5752

    accuracy                           0.77     11520
   macro avg       0.78      0.77      0.77     11520
weighted avg       0.78      0.77      0.77     11520


ادخل تغريدة جديدة (اكتب 'exit' للخروج):
>>> كيف حالك
➡️ التغريدة سلبية

ادخل تغريدة جديدة (اكتب 'exit' للخروج):
>>> exit
تم إنهاء البرنامج.


In [4]:
# حفظ النموذج و TF-IDF Vectorizer
import joblib
joblib.dump(mlp, "mlp_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("تم حفظ النموذج وVectorizer جاهزين للاستخدام في الويب!")


تم حفظ النموذج وVectorizer جاهزين للاستخدام في الويب!


In [1]:
# 9️⃣ واجهة تجربة التغريدات الجديدة
while True:
    tweet = input("\nادخل تغريدة جديدة (اكتب 'exit' للخروج):\n>>> ")
    if tweet.lower() == "exit":
        print("تم إنهاء البرنامج.")
        break
    tweet_clean = clean_text(tweet)
    tweet_vector = vectorizer.transform([tweet_clean])
    pred = mlp.predict(tweet_vector)[0]
    print("➡️ التغريدة إيجابية" if pred == 1 else "➡️ التغريدة سلبية")



ادخل تغريدة جديدة (اكتب 'exit' للخروج):
>>> انت كلب


NameError: name 'clean_text' is not defined

In [2]:
# 9️⃣ واجهة تجربة التغريدات الجديدة
while True:
    tweet = input("\nادخل تغريدة جديدة (اكتب 'exit' للخروج):\n>>> ")
    if tweet.lower() == "exit":
        print("تم إنهاء البرنامج.")
        break
    tweet_clean = clean_text(tweet)
    tweet_vector = vectorizer.transform([tweet_clean])
    pred = mlp.predict(tweet_vector)[0]
    print("➡️ التغريدة إيجابية" if pred == 1 else "➡️ التغريدة سلبية")



ادخل تغريدة جديدة (اكتب 'exit' للخروج):
>>> انت جميل


NameError: name 'clean_text' is not defined